##### 过滤步骤：1.先用四大过滤原则对每个数据集过滤 2.ChEMBL用NIST过滤，HMDB和chembl用和NIST过滤

In [2]:
nist_train = open('../data/fastei/nist/train_set_smiles.txt') 
smiles_train = nist_train.readlines()
smiles_train = [smi.strip() for smi in smiles_train]
nist_val = open('../data/fastei/nist/validation_set_smiles.txt')
smiles_val = nist_val.readlines()
smiles_val = [smi.strip() for smi in smiles_val]
nist_test = open('../data/fastei/nist/test_set_smiles.txt') 
smiles_test = nist_test.readlines()
smiles_test = [smi.strip() for smi in smiles_test]

print(len(smiles_train)) # 237108
print(len(smiles_val)) # 11599
print(len(smiles_test)) # 11600
print(len(set(smiles_train)&set(smiles_test))) 
print(len(set(smiles_train)&set(smiles_val)))
print(len(set(smiles_val)&set(smiles_test)))

237108
11599
11600
0
0
0


In [ ]:
from rdkit import Chem


suppl_chem = Chem.SDMolSupplier('../data/fastei/chembl/chembl_28.sdf')
mols_chembl = [m for m in suppl_chem if m] # 2066376
print(len(mols_chembl)) # 
chembl_smi = [Chem.MolToSmiles(mol) for mol in mols_chembl] # 2066376
print(len(chembl_smi))

In [ ]:
from rdkit import Chem


suppl_h = Chem.SDMolSupplier('../data/fastei/hmdb/structures.sdf')
hmdb_mols = [mol for mol in suppl_h if mol] 
print(len(hmdb_mols)) # 217766
hmdb_smi = [mol.GetProp('SMILES') for mol in hmdb_mols]
print(len(hmdb_smi)) # 217766

In [ ]:
from utils.filter import filter_nist

f_mols_n, nist_unq, nist_smi = filter_nist(smiles_train+smiles_val+smiles_test)
print(len(nist_smi)) # 255820,4487

In [ ]:
from utils.filter import filter_yq

f_chembl, chembl_unq, chembl_qua_smi = filter_yq(chembl_smi)

In [ ]:
from utils.filter import filter_yq

f_hmdb, hmdb_unq, hmdb_qua_smi = filter_yq(hmdb_smi)

In [ ]:
f_chembl_mols = []
for mol, smi in zip(f_chembl, chembl_qua_smi):
    if smi in nist_smi:
        continue
    else:
        f_chembl_mols.append(mol)
print(len(f_chembl_mols)) 

In [18]:
f_hmdb_mols = []
for mol, smi in zip(f_hmdb, hmdb_qua_smi):
    if smi in nist_smi or smi in chembl_qua_smi:
        continue
    else:
        f_hmdb_mols.append(mol)
print(len(f_hmdb_mols)) # 118489

118489


##### 过滤完成之后生成sdf文件，之后用NEIMS生成预测谱，然后将intensities归一化

In [ ]:
from rdkit.Chem import AllChem 

writer = AllChem.SDWriter('../data/fastei/chembl/f_chembl.sdf')
for mol in f_chembl_mols:
    writer.write(mol)
writer.close()

In [19]:
from rdkit.Chem import AllChem 

writer = AllChem.SDWriter('../data/fastei/hmdb/f_hmdb.sdf')
for mol in f_hmdb_mols:
    writer.write(mol)
writer.close()